# Introducción a las librerías de Python para desarrollo web


Un servidor web es un software o un dispositivo que se encarga de recibir las solicitudes de los clientes (generalmente navegadores web) y proporcionarles una respuesta. La respuesta puede ser una página web, una imagen, un archivo, etc. El servidor web también puede ejecutar scripts y aplicaciones en el lado del servidor, como aplicaciones de PHP o Python, para generar dinámicamente el contenido que se envía al cliente.

Existen multitud de servicios y servidores web que podemos utilizar, como Django, Flask...

Nosotros utilizaremos Tornado. 

Tornado es un framework de servidor web de código abierto escrito en Python. Fue desarrollado por la empresa FriendFeed (ahora propiedad de Facebook) y es utilizado en muchas aplicaciones web de alto rendimiento y escalabilidad.

Tornado es especialmente adecuado para aplicaciones web que requieren un gran número de conexiones simultáneas, como aplicaciones de chat, juegos en línea, aplicaciones web en tiempo real y aplicaciones de streaming de datos.

Una de las características más importantes de Tornado es su motor de I/O no bloqueante, que permite manejar un gran número de conexiones simultáneas sin consumir demasiados recursos del sistema. Tornado también tiene una API simple y fácil de usar para el desarrollo de aplicaciones web y admite la programación asíncrona, lo que permite que las aplicaciones manejen varias tareas simultáneamente.

## Instalación

Para utilizar Tornado, necesitaremos su biblioteca llamada tornado. Para instalarla podemos ejecutar esta línea

`$ pip install tornado`

Si aparece un error, podríamos ejecutar

`$ pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org tornado`

o bien:

`$ pip install --user --trusted-host pypi.org --trusted-host files.pythonhosted.org tornado`

Necesitaremos también crear unas páginas web que serviremos en un futuro

Para comenzar haremos una web de ejemplo:

In [ ]:
# Do not run code!!! copy and paste code into server.py and run through console!

import tornado.ioloop 
import tornado.web

class MainHandler(tornado.web.RequestHandler):
    def get(self):
        self.render("html\index.html")

def make_app():
    return tornado.web.Application([
        (r"/", MainHandler)
    ])

if __name__ == "__main__":
    app = make_app()
    app.listen(8888)
    print("Server started on port 8888")
    tornado.ioloop.IOLoop.current().start()

## Manejando peticiones GET y POST

Las peticiones GET y POST son dos métodos HTTP (Hypertext Transfer Protocol) utilizados para solicitar información a un servidor web.

- La petición GET se utiliza para obtener información del servidor. Es el método HTTP más común y se utiliza para solicitar una página web o un recurso específico, como una imagen o un archivo. La información se envía en la línea de dirección de la URL (Uniform Resource Locator) y es visible para el usuario. Por lo general, las peticiones GET son seguras y no tienen efectos secundarios, ya que solo obtienen información y no la modifican.

- La petición POST se utiliza para enviar información al servidor. Se utiliza para enviar datos de un formulario web, subir un archivo o realizar una operación que cambie el estado del servidor. La información se envía en el cuerpo de la solicitud y no es visible para el usuario. Las peticiones POST son menos seguras que las GET ya que pueden tener efectos secundarios, como actualizar o eliminar información en el servidor.

Por ejemplo:

In [ ]:
# Do not run code!!! copy and paste code into server.py and run through console!

import tornado.ioloop 
import tornado.web

class MainHandler(tornado.web.RequestHandler):
    def get(self):
        self.render("html\index.html")
        print("served")
    def post(self):
        searchterm = self.get_argument('searchterm')
        print(searchterm)
        self.write(searchterm)
        

def make_app():
    return tornado.web.Application([
        (r"/", MainHandler)
    ])

if __name__ == "__main__":
    app = make_app()
    app.listen(8888)
    print("Server started on port 8888")
    tornado.ioloop.IOLoop.current().start()

El servidor puede gestionar automaticamente las peticiones de manera estática, por ejemplo:

In [ ]:
# Do not run code!!! copy and paste code into server.py and run through console!

import tornado.ioloop 
import tornado.web

class MainHandler(tornado.web.RequestHandler):
    def get(self):
        self.render("html/index.html")
        print("served")
    def post(self):
        searchterm = self.get_argument('searchterm')
        print(searchterm)
        self.write(searchterm)

def make_app():
    return tornado.web.Application([
        (r"/", MainHandler),
        (r"/html/(.*)", tornado.web.StaticFileHandler, {"path": "html"}), # Sirve las páginas HTML
        (r"/resources/(.*)", tornado.web.StaticFileHandler, {"path": "resources"}), # sirve los recursos
    ])

if __name__ == "__main__":
    app = make_app()
    app.listen(8888)
    print("Server started on port 8888")
    tornado.ioloop.IOLoop.current().start()

Para pasar argumentos a la hora de renderizar la web, utilizamos keyword arguments y en el HTML recuperamos el valor utilizando "{{ }}", por ejemplo:

In [ ]:
# Do not run code!!! copy and paste code into server.py and run through console!

import tornado.ioloop 
import tornado.web

class MainHandler(tornado.web.RequestHandler):
    def get(self):
        self.render("html/index.html")
        print("served")
    def post(self):
        searchterm = self.get_argument('searchterm')
        print(searchterm)
        self.render("html/searchpage.html", searchterm=searchterm) # Cuando recibamos un término de búsqueda servimos la web. Aquí podríamos psasr toda la información que quisieramos.
        

def make_app():
    return tornado.web.Application([
        (r"/", MainHandler),
        (r"/html/(.*)", tornado.web.StaticFileHandler, {"path": "html"}), # Nos hace la vida más sencilla 
        (r"/resources/(.*)", tornado.web.StaticFileHandler, {"path": "resources"}), # sirve los recursos
    ])

if __name__ == "__main__":
    app = make_app()
    app.listen(8888)
    print("Server started on port 8888")
    tornado.ioloop.IOLoop.current().start()

En HTML simplemente: 

In [ ]:
<h2>Mostrando resultados de: {{searchterm}}</h2>

Lo que hemos conseguido es: 


![img](resources\1.gif "img")


Hasta ahora tenemos lo necesario para crear una aplicación web. Este framework también puede usar para crear rápidamente una API REST de esta manera:

In [ ]:
# Do not run code!!! copy and paste code into server.py and run through console!

import tornado.ioloop 
import tornado.web
import json



class MainHandler(tornado.web.RequestHandler):
    def get(self):
        self.render("html/index.html")
    def post(self):
        searchterm = self.get_argument('searchterm')
        self.render("html/searchpage.html", searchterm=searchterm)
        
class ApiHandler(tornado.web.RequestHandler):
    def get(self,data):
        vehiculos = ""
        with open("resources/vehiculos.json", "r") as f:
            print(data)
            vehiculos = json.dumps(json.load(f))
            self.write(vehiculos)
            

def make_app():
    return tornado.web.Application([
        (r"/", MainHandler),
        (r"/vehiculos/(.*)",ApiHandler),
        (r"/html/(.*)", tornado.web.StaticFileHandler, {"path": "html"}), # Nos hace la vida más sencilla 
        (r"/resources/(.*)", tornado.web.StaticFileHandler, {"path": "resources"}), # sirve los recursos
    ])

if __name__ == "__main__":
    app = make_app()
    app.listen(8888)
    print("Server started on port 8888")
    tornado.ioloop.IOLoop.current().start()

y en el html incluimos este script:

In [ ]:
    var url = "http://" + window.location.host + "/vehiculos/"+'{{searchterm}}'
    
    async function exampleFetch(url) {
        const response = await fetch(url);
        const json = await response.text();
        var vehiculos = await JSON.parse(json);
        console.log(vehiculos);
    }    
    console.log(url);

    exampleFetch(url)

Podemos utilizar el formulario POST anterior para hacer una consulta GET usando la API REST, de esta forma:

In [ ]:
# Do not run code!!! copy and paste code into server.py and run through console!

import tornado.ioloop
import tornado.web
import json


class MainHandler(tornado.web.RequestHandler):
    def get(self):
        self.render("html/index.html")

    def post(self):
        searchterm = self.get_argument('searchterm')
        self.render("html/searchpage.html", searchterm=searchterm)


class ApiHandler(tornado.web.RequestHandler):
    def get(self, data):
        palabras = data.split()
        print(palabras)
        resultados = []
        with open("resources/vehiculos.json", "r", encoding="utf-8-sig") as f:
            vehiculos = json.load(f)
            vehiculos = vehiculos['vehiculos']
            for vehiculo in vehiculos:
                for palabra in palabras:
                    if( palabra.lower() in [vehiculo['marca'].lower(),vehiculo['modelo'].lower(),vehiculo['combustible'].lower(), str(vehiculo["año"])]):
                        resultados.append(vehiculo)
        self.write(json.dumps(resultados))


def make_app():
    return tornado.web.Application([
        (r"/", MainHandler),
        (r"/vehiculos/(.*)", ApiHandler),
        (r"/html/(.*)", tornado.web.StaticFileHandler,{"path": "html"}),  # Nos hace la vida más sencilla
        (r"/resources/(.*)", tornado.web.StaticFileHandler,{"path": "resources"}),  # sirve los recursos
    ])


if __name__ == "__main__":
    app = make_app()
    app.listen(8888)
    print("Server started on port 8888")
    tornado.ioloop.IOLoop.current().start()


Y editamos un poco el script para mostrar una tabla con los resultados de la búsqueda, por ejemplo:

In [ ]:
var url = "http://" + window.location.host + "/vehiculos/" + '{{searchterm}}'
async function exampleFetch(url) {

    const response = await fetch(url);
    const json = await response.text();
    var vehiculos = await JSON.parse(json);
    console.log(vehiculos);

    result = document.getElementById('log')

    html = '<table class="table table-striped"><thead><tr><th>Marca</th><th>Modelo</th><th>Año</th><th>Kilometros</th><th>Combustible</th><th>Descripción</th></tr></thead><tbody>'

    vehiculos.forEach(vehiculo => {
        html += '<tr><td>' + vehiculo['marca'] + '</td><td>' + vehiculo['modelo'] + '</td><td>' + vehiculo['año'] + '</td><td>' + vehiculo['kilometros'] + '</td><td>' + vehiculo['combustible'] + '</td><td>' + vehiculo['descripción'] + '</td></tr>'
    });

    html += "</tbody> </table>"

    result.innerHTML = html


}
console.log(url);

exampleFetch(url)

En resumen, lo que realizamos es lo siguiente:

![img](resources\2.gif "img")

Evidentemente esto es una ejemplo de gestión de las peticiones GET y POST y esto se puede realizar de muchas otras formas más cómodas.

Esta ha sido una introducción básica al desarrollo web en Python usando Tornado. En Tornado se pueden configurar cookies, certificados HTTPS y añadir multitud de capacidades, manteniendo un diseño simple, legible y mantenible.

¡Empecemos la aventura de crear aplicaciones web poderosas y escalables juntos!